In [4]:
from langchain.prompts import PromptTemplate
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_community.utilities import ZapierNLAWrapper
from langchain_community.agent_toolkits import ZapierToolkit
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from dotenv import load_dotenv
load_dotenv()

True

# 1. Using Automation as LLM Tool: Zapier
There are other automation providers. This is an overview:
- Use **Zapier** for ease of use and extensive app integrations.
- Use **Integromat (Make)** for advanced, detailed workflows with a visual editor.
- Use **Microsoft Power Automate** if you are embedded in the Microsoft ecosystem and need RPA.
- Use **IFTTT** for simple, personal, and smart home automations.
- Use **Tray.io** for enterprise-grade, complex automation needs.
- Use **Automate.io** for straightforward business process automation.
- Use **Workato** for secure, scalable, and real-time enterprise automation.
- Use **n8n** for open-source flexibility and custom workflows.
- Use **Pabbly Connect** for cost-effective, unlimited automation options.
- Use **Ansible** for IT and DevOps automation, including infrastructure provisioning, configuration management, and application deployment.

Here is how to use Zapier as an LLM tool (with langchain's wrapper). For a more detailed explanation, check the files *app.py* (for UI) and *prompt_generation.py*

In [5]:
# Model-related instantiations
llm = ChatOpenAI(temperature=1., model_name="gpt-4o")

# Get the tools
zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
tools = toolkit.tools

# Initialize agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Init whisper
model_whisper_client = OpenAI()

# Define the prompt template
prompt_template = PromptTemplate(
    template="""
        Send an email to {receiver} via gmail summarizing the following text: '''{text}'''
    """,
    input_variables=["receiver", "text"]
)


def send_email(self, file_voice, receiver_email: str) -> None:
    # Transcribe file
    text = model_whisper_client.audio.transcriptions.create(
        model="whisper-1",
        file=file_voice
    ).text

    # Send email using zapier
    self.agent_executor.invoke({"input": self.prompt_template.format(receiver=receiver_email, text=text)})

In [2]:
# here you can use 'send_mail' if you have receiver_email and file_voice